In [2]:
import simpy
import random

class VirtualMachine:
    def __init__(self, env, id, processing_capability, num_cpus, ram, storage_capacity, data_center):
        self.env = env
        self.id = id
        self.processing_capability = processing_capability
        self.num_cpus = num_cpus
        self.ram = ram
        self.storage_capacity = storage_capacity
        self.data_center = data_center
        self.data_access_log = {}

    def process_task(self, task, target_dc, data_transfer_time=0):
        start_time = self.env.now
        total_processing_time = task['size'] / self.processing_capability + data_transfer_time
        yield self.env.timeout(total_processing_time)
        end_time = self.env.now

        # Calculate response time and check for SLA violation or near violation
        response_time = end_time - start_time
        sla_rt = 180  # SLORT in seconds
        w = 0.8  # Define the weight factor
        th_rt = w * sla_rt  # Threshold for near SLA violation

        penalty = 0
        near_violation = False
        if response_time > sla_rt:
            penalty = 0.0025  # Penalty per violation
        elif response_time > th_rt:
            near_violation = True  # Task is near to causing SLA violation

        cost = self.data_center.calculate_cost(task['size'], target_dc) - penalty
        self.data_center.total_revenue += (0.7 - penalty)  # Update total revenue for the data center
        self.data_center.near_violations += int(near_violation)  # Update near violations count for the data center

        self.data_access_log[task['id']] = {
            'response_time': response_time,
            'penalty': penalty,
            'near_violation': near_violation
        }


class DataCenter:
    def __init__(self, env, id, num_vms, vm_specs, pricing, bw_params):
        self.env = env
        self.id = id
        self.vm_specs = vm_specs
        self.pricing = pricing
        self.bw_params = bw_params
        self.total_revenue = 0
        self.near_violations = 0
        self.vms = [VirtualMachine(env, f"VM_{id}_{i}", **vm_specs, data_center=self) for i in range(num_vms)]
    
    def get_total_revenue(self):
        return self.total_revenue
    
    def get_near_violations_count(self):
        return self.near_violations
    
    def calculate_cost(self, task_size, target_dc):
        cpu_cost = task_size / 107 * self.pricing['cpu_price']
        storage_cost = self.vm_specs['storage_capacity'] * self.pricing['storage_price']

        if self.id == target_dc.id:
            # Intra-DC transfer
            bw_cost = task_size * self.pricing['bw_price']['intra_dc']
        elif self.id.split('_')[1] == target_dc.id.split('_')[1]:
            # Intra-region transfer
            bw_cost = task_size * self.pricing['bw_price']['intra_region']
        else:
            # Inter-region transfer
            bw_cost = task_size * self.pricing['bw_price']['inter_region']

        return cpu_cost + storage_cost + bw_cost

    def calculate_data_transfer_time(self, task_size, target_dc):
        if self.id == target_dc.id:
            # Intra-DC transfer
            bw_capacity = self.bw_params['intra_dc']['capacity']
            bw_delay = self.bw_params['intra_dc']['delay']
        elif self.id.split('_')[1] == target_dc.id.split('_')[1]:
            # Intra-region transfer
            bw_capacity = self.bw_params['intra_region']['capacity']
            bw_delay = self.bw_params['intra_region']['delay']
        else:
            # Inter-region transfer
            bw_capacity = self.bw_params['inter_region']['capacity']
            bw_delay = self.bw_params['inter_region']['delay']

        data_transfer_time = task_size / bw_capacity + bw_delay / 1000  # Convert delay from ms to seconds
        return data_transfer_time

class CloudProvider:
    def __init__(self, env, id, num_regions, dcs_per_region, vm_specs, pricing, bw_params):
        self.env = env
        self.id = id
        self.pricing = pricing
        self.bw_params = bw_params
        region_keys = list(pricing.keys())  # Get the region keys from pricing
        self.regions = [DataCenter(env, f"DC_{id}_{region_keys[r]}", random.randint(*dcs_per_region), vm_specs, pricing[region_keys[r]], bw_params) for r in range(num_regions)]

    def find_target_dc(self, task):
        # Simplified logic: Randomly select a data center
        return random.choice(self.regions)

# Bandwidth parameters
bw_params = {
    'inter_region': {'capacity': 500, 'delay': 150},
    'intra_region': {'capacity': 1000, 'delay': 50},
    'intra_dc': {'capacity': 8000, 'delay': 10}
}

# Simulation parameters
num_providers = 3
num_regions = 3
dcs_per_provider = (2, 5)
num_vms_per_dc = 8
vm_specs = {
    'processing_capability': 1500,  # in MIPS
    'num_cpus': 2,
    'ram': 4,  # in Gb
    'storage_capacity': 8  # in Gb
}
num_data = 200
task_size_range = (200, 1000)  # Task size range in MI

# Pricing structure
pricing = {
    'Provider_1': {
        'US': {'cpu_price': 0.020, 'storage_price': 0.006, 'bw_price': {'intra_dc': 0.001, 'inter_region': 0.008}},
        'EU': {'cpu_price': 0.025, 'storage_price': 0.006, 'bw_price': {'intra_dc': 0.0015, 'inter_region': 0.008}},
        'AS': {'cpu_price': 0.027, 'storage_price': 0.0066, 'bw_price': {'intra_dc': 0.002, 'inter_region': 0.008}}
    },
    'Provider_2': {
        'US': {'cpu_price': 0.020, 'storage_price': 0.0096, 'bw_price': {'intra_dc': 0.001, 'inter_region': 0.008}},
        'EU': {'cpu_price': 0.018, 'storage_price': 0.008, 'bw_price': {'intra_dc': 0.0015, 'inter_region': 0.008}},
        'AS': {'cpu_price': 0.020, 'storage_price': 0.0096, 'bw_price': {'intra_dc': 0.002, 'inter_region': 0.008}}
    },
    'Provider_3': {
        'US': {'cpu_price': 0.0095, 'storage_price': 0.0012, 'bw_price': {'intra_dc': 0.001, 'inter_region': 0.008}},
        'EU': {'cpu_price': 0.0090, 'storage_price': 0.0096, 'bw_price': {'intra_dc': 0.0015, 'inter_region': 0.008}},
        'AS': {'cpu_price': 0.0080, 'storage_price': 0.0090, 'bw_price': {'intra_dc': 0.002, 'inter_region': 0.008}}
    }
}

env = simpy.Environment()

providers = []
for i in range(num_providers):
    provider_key = f'Provider_{i+1}'  # This should match the keys in the pricing dictionary
    if provider_key in pricing:
        provider = CloudProvider(env, provider_key, num_regions, dcs_per_provider, vm_specs, pricing[provider_key], bw_params)
        providers.append(provider)
    else:
        print(f"Warning: Pricing not found for {provider_key}")
        
tasks = [{'id': i, 'size': random.randint(*task_size_range)} for i in range(num_data)]

for task in tasks:
    # Example logic for selecting source and target data centers
    random_provider = random.choice(providers)
    source_dc = random.choice(random_provider.regions)
    target_dc = random.choice(random_provider.regions)
    data_transfer_time = source_dc.calculate_data_transfer_time(task['size'], target_dc)
    random_vm = random.choice(target_dc.vms)
    env.process(random_vm.process_task(task, target_dc, data_transfer_time))

env.run()